In [1]:
import json, nltk, io, pickle
import numpy as np
from itertools import chain

In [34]:
dev8_char = np.load('/pio/data/data/squad/glove840B/dev_char_ascii.pkl')

In [37]:
fixed = []

for q, x in dev8_char:
    q_fix = list(chain(*q))
    x_fix = list(chain(*x))
    fixed.append([q_fix, x_fix])

In [39]:
with open('/pio/data/data/squad/glove840B/dev_char_ascii.pkl', 'w') as f:
    pickle.dump(fixed, f)

### Read data

In [88]:
with io.open('/pio/data/data/squad/dev-v1.1.json', 'r', encoding='utf-8') as f:
    dev = json.load(f)

with io.open('/pio/data/data/squad/train-v1.1.json', 'r', encoding='utf-8') as f:
    train = json.load(f)

In [8]:
glove6B = np.load('/pio/data/data/glove_vec/6B/glove/glove.6B.wordlist.pkl')
glove840B = np.load('/pio/data/data/glove_vec/840B/glove/glove.840B.wordlist.pkl')

In [69]:
w_to_i_840B = {glove840B[i] : i for i in xrange(len(glove840B))}

In [26]:
train6B = np.load('/pio/data/data/squad/glove6B/train_words.pkl')

In [134]:
dev6B = np.load('/pio/data/data/squad/glove6B/dev_words.pkl')

In [89]:
data = []

for par in train['data']:    
    for con in par['paragraphs']:
        context = con['context']
        context_tok = nltk.word_tokenize(context)
        
        for q in con['qas']:
            question = q['question']
            question_tok = nltk.word_tokenize(question)
                   
            data.append([question_tok, context_tok])

In [132]:
data_dev = []

for par in dev['data']:    
    for con in par['paragraphs']:
        context = con['context']
        context_tok = nltk.word_tokenize(context)
        
        for q in con['qas']:
            question = q['question']
            question_tok = nltk.word_tokenize(question)
                   
            data_dev.append([question_tok, context_tok])

In [96]:
def get_num(w):
    return w_to_i_840B.get(w, 0)

train = []

for i, (q, x) in enumerate(data):
    ans = train6B[i][0]
    q_num = map(get_num, q)
    x_num = map(get_num, x)
    train.append([ans, q_num, x_num])

In [139]:
dev = []

for i, (q, x) in enumerate(data_dev):
    ans = dev6B[i][0]
    q_num = map(get_num, q)
    x_num = map(get_num, x)
    dev.append([ans, q_num, x_num])

In [116]:
def make_bin_feats(sample):
    q, x = sample
    qset = set(q)
    return [w in qset for w in x]

In [117]:
train_bin = map(make_bin_feats, data)

In [143]:
dev_bin = map(make_bin_feats, data_dev)

In [ ]:
# 0 - unk
# 1 - start
# 2 - end
# 3 - not_a_word char (added later, in wikipedia negative samples)
# there are no 1s or 2s in data, so these are safe

chars = [unichr(i) for i in xrange(128)]
c_to_i = {chars[i] : i for i in xrange(128)}

def get_char_nums_for_word(w):
    return [[1] + [c_to_i.get(c, 0) for c in w] + [2]]

In [33]:
'''
wynik char jest zly, trzeba usunac jeden wymiar (fix na powyzej)
'''

'\nwynik char jest zly, trzeba usunac jeden wymiar\n'

In [124]:
train_char = []

for q, x in data:
    q_char = map(get_char_nums_for_word, q)
    x_char = map(get_char_nums_for_word, x)
    train_char.append([q_char, x_char])

In [144]:
dev_char = []

for q, x in data_dev:
    q_char = map(get_char_nums_for_word, q)
    x_char = map(get_char_nums_for_word, x)
    dev_char.append([q_char, x_char])

In [122]:
with open('/pio/data/data/squad/glove840B/train_words.pkl', 'w') as f:
    pickle.dump(train, f)

with open('/pio/data/data/squad/glove840B/train_bin_feats.pkl', 'w') as f:
    pickle.dump(train_bin, f)
    
with open('/pio/data/data/squad/glove840B/train_char_ascii.pkl', 'w') as f:
    pickle.dump(train_char, f)

In [146]:
with open('/pio/data/data/squad/glove840B/dev_words.pkl', 'w') as f:
    pickle.dump(dev, f)

with open('/pio/data/data/squad/glove840B/dev_bin_feats.pkl', 'w') as f:
    pickle.dump(dev_bin, f)
    
with open('/pio/data/data/squad/glove840B/dev_char_ascii.pkl', 'w') as f:
    pickle.dump(dev_char, f)